In [1]:
from app.dao.db_base import engine, async_session_maker
from app.models.stock import Stock
from app.models.item import Item
from app.models.cat import Cat
from app.models.user import User

from sqlalchemy import select, func

In [2]:
async def find_all_stocks():
    async with async_session_maker() as session:
        query = select(Stock)
        try:
            result = await session.execute(query)
            return result.scalars().all()
        except Exception as e:
            print(e)
            raise

In [3]:
# async def get_stocks():
#     return await find_all_stocks()

In [4]:
stocks = await find_all_stocks()
print(stocks)

2025-04-11 15:45:18,796 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-11 15:45:18,826 INFO sqlalchemy.engine.Engine SELECT stocks.reference, stocks.date, stocks.consignor, stocks.ready, stocks.note, stocks.id 
FROM stocks
2025-04-11 15:45:18,827 INFO sqlalchemy.engine.Engine [generated in 0.00133s] ()
2025-04-11 15:45:18,830 INFO sqlalchemy.engine.Engine ROLLBACK
[<app.models.stock.Stock object at 0x7f9b73f8cb20>, <app.models.stock.Stock object at 0x7f9b73f8cd00>, <app.models.stock.Stock object at 0x7f9b73f8cd30>, <app.models.stock.Stock object at 0x7f9b73f8cd60>, <app.models.stock.Stock object at 0x7f9b73f8cd90>, <app.models.stock.Stock object at 0x7f9b73f8cdc0>, <app.models.stock.Stock object at 0x7f9b73f8ccd0>, <app.models.stock.Stock object at 0x7f9b73f8ce20>]


In [5]:
from app.schemas.stock import Stock as StockSchema
print([StockSchema(**stock.to_dict()) for stock in stocks])

[Stock(reference='FEU034/25', date=datetime.datetime(2025, 2, 2, 0, 0), consignor='Pujol', ready=False, note='tech note'), Stock(reference='FEU036/25', date=datetime.datetime(2025, 2, 10, 0, 0), consignor='Pujol', ready=False, note='tech note'), Stock(reference='FEU039/25', date=datetime.datetime(2025, 2, 13, 0, 0), consignor='Pujol', ready=False, note='tech note'), Stock(reference='FEU040/25', date=datetime.datetime(2025, 2, 14, 20, 53, 26, 999000), consignor='Puijol', ready=False, note='Postman note'), Stock(reference='FEU041/25', date=datetime.datetime(2025, 2, 14, 20, 53, 26, 999000), consignor='Puijol', ready=False, note='Postman note'), Stock(reference='FEU042/25', date=datetime.datetime(2025, 2, 14, 20, 53, 26, 999000), consignor='Puijol', ready=False, note='Postman note'), Stock(reference='FEU049/25', date=datetime.datetime(2025, 2, 16, 0, 0), consignor='Pujol 2', ready=False, note='Postman note'), Stock(reference='FE052/25', date=datetime.datetime(2025, 4, 1, 0, 0), consignor=

In [6]:
from sqlalchemy.orm import joinedload
async def get_full_item():
    async with async_session_maker() as session:
        query = select(Item).options(joinedload(Item.stock))
        try:
            result = await session.execute(query)

            return result.scalars().all()
        except Exception as e:
            print(e)
            raise

In [7]:
items = await get_full_item()

2025-04-11 15:45:19,011 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-11 15:45:19,018 INFO sqlalchemy.engine.Engine SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id, stocks_1.reference, stocks_1.date, stocks_1.consignor, stocks_1.ready, stocks_1.note AS note_1, stocks_1.id AS id_1 
FROM items LEFT OUTER JOIN stocks AS stocks_1 ON stocks_1.id = items.stock_id
2025-04-11 15:45:19,018 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ()
2025-04-11 15:45:19,022 INFO sqlalchemy.engine.Engine ROLLBACK


In [8]:
print(items)

[<app.models.item.Item object at 0x7f9b73f8eb30>, <app.models.item.Item object at 0x7f9b73f8cf40>, <app.models.item.Item object at 0x7f9b73f8cee0>, <app.models.item.Item object at 0x7f9b73f8d090>, <app.models.item.Item object at 0x7f9b73f8cfd0>]


In [9]:
from pydantic import BaseModel, ConfigDict
from datetime import datetime

class Stock(BaseModel):
    reference: str
    date: datetime
    consignor: str
    ready: bool

class Item(BaseModel):
    id: int
    lot: str
    pallet: str
    roll: str
    note: str | None = None
    stock_id: int | None = None
    cat_id: int
    load_id: int | None = None
    # stock: Stock
    reference: str
    date: datetime
    consignor: str
    ready:bool

    model_config = ConfigDict(from_attributes=True)

In [10]:
for item in items:
   print( item.__dict__)
   if item.stock:
         print(item.stock.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f9b73f19960>, 'load_id': None, 'cat_id': 3, 'pallet': '0034', 'note': 'Item note', 'stock_id': 1, 'lot': '00245', 'roll': '0101', 'id': 1, 'stock': <app.models.stock.Stock object at 0x7f9b73f8c940>}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f9b73f199c0>, 'consignor': 'Pujol', 'note': 'tech note', 'ready': False, 'reference': 'FEU034/25', 'date': datetime.datetime(2025, 2, 2, 0, 0), 'id': 1}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f9b73f19a20>, 'load_id': None, 'cat_id': 3, 'pallet': '0034', 'note': 'Item note', 'stock_id': 2, 'lot': '00245', 'roll': '0103', 'id': 2, 'stock': <app.models.stock.Stock object at 0x7f9b73f8d000>}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f9b73f19a80>, 'consignor': 'Pujol', 'note': 'tech note', 'ready': False, 'reference': 'FEU036/25', 'date': datetime.datetime(2025, 2, 10, 0, 0), 'id': 2}
{'_sa_i

In [11]:
Item.model_validate(items[0])

ValidationError: 4 validation errors for Item
reference
  Field required [type=missing, input_value=<app.models.item.Item object at 0x7f9b73f8eb30>, input_type=Item]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
date
  Field required [type=missing, input_value=<app.models.item.Item object at 0x7f9b73f8eb30>, input_type=Item]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
consignor
  Field required [type=missing, input_value=<app.models.item.Item object at 0x7f9b73f8eb30>, input_type=Item]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
ready
  Field required [type=missing, input_value=<app.models.item.Item object at 0x7f9b73f8eb30>, input_type=Item]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [12]:
for item in items:
    print(dict(item))

TypeError: 'Item' object is not iterable